In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import joblib
from copy import deepcopy

# Compute engine stand alone

In [4]:
file = joblib.load('schemas.barfi')

In [6]:
file['entradas']

{'nodes': [{'type': 'constante',
   'id': 'node_17214380805906',
   'name': 'constante-1',
   'options': [['valor', 1]],
   'state': {},
   'interfaces': [['saida', {'id': 'ni_17214380805907', 'value': None}]],
   'position': {'x': 29.61496596134924, 'y': 120.81975240581971},
   'width': 200,
   'twoColumn': False,
   'customClasses': ''},
  {'type': 'constante',
   'id': 'node_17214380832128',
   'name': 'constante-2',
   'options': [['valor', 2]],
   'state': {},
   'interfaces': [['saida', {'id': 'ni_17214380832129', 'value': None}]],
   'position': {'x': 26.0153194879953, 'y': 262.4058470244086},
   'width': 200,
   'twoColumn': False,
   'customClasses': ''},
  {'type': 'entrada',
   'id': 'node_172143808522910',
   'name': 'entrada-1',
   'options': [],
   'state': {},
   'interfaces': [['entrada', {'id': 'ni_172143808523011', 'value': None}],
    ['saida', {'id': 'ni_172143808523012', 'value': None}]],
   'position': {'x': 283.9899834116954, 'y': 98.02199140791136},
   'width': 

In [4]:
from_client = joblib.load('_from_client.pkl')

In [5]:
from blocks import init
from streamlit import session_state as session
session.dados = pd.DataFrame()
init()

2024-07-20 20:03:06.429 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


import blocks
starting blocks
colums: RangeIndex(start=0, stop=0, step=1)


In [6]:
from Barfi import ComputeEngine

def get_execution(fluxo, base_blocks_list):
    _ce = ComputeEngine(blocks=base_blocks_list)
    _ce.add_editor_state(fluxo)
    _ce._map_block_link()
    _ce._execute_compute()

In [7]:
get_execution(file['soma3'], session.blocks)

Resultado: 3


# Entrada block

In [31]:
file = joblib.load('schemas.barfi')
entr = file['entradas']

In [54]:
for fluxo in file:
    print(fluxo)

teste
teste4
soma3
entradas


In [32]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


In [33]:
def get_dict_from_interfaces(interface):
    aux = {}
    for i in interface:
        aux[i[0]] = i[1]
    return aux


def encode_dict_to_interface(aux_dict):
    interface = [[key, aux_dict[key]] for key in aux_dict]
    return interface


def encode_val_to_entradas(entra, val_dict):
    aux = entra.copy()
    for entrada in aux:
        name = entrada['name']
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        tmp['entrada']['value'] = val_dict[name]
        encoded_interface = encode_dict_to_interface(tmp)
        entrada['interfaces'] = encoded_interface
    return aux

def get_ids_entradas(entradas):
    aux = entradas.copy()
    ids = []
    for entrada in aux:
        tmp = get_dict_from_interfaces(entrada['interfaces'])
        ids.append(tmp['entrada']['id'])
    return ids


def get_ids_from_conn(connections, to_ids):
    aux = connections.copy()
    entry_dict = {}
    ids = []
    ids_from = []
    pos = []
    for i, conn in enumerate(connections):
        if conn['to'] in to_ids:
            entry_dict[i] = {'id': conn['id'],
                             'from': conn['from'],
                             'to': conn['to']}
    return entry_dict


def encode_entradas_to_const(entra, val_dict):
    aux = entra['nodes'].copy()
    for entrada in aux:
        name = entrada['name']
        if name in val_dict:
            entrada['type'] = 'constante'
            entrada['options'] = [['valor', val_dict[name]]]
            tmp = get_dict_from_interfaces(entrada['interfaces'])
            del tmp['entrada']
            encoded_interface = encode_dict_to_interface(tmp)
            entrada['interfaces'] = encoded_interface


def get_to_dict_from_conn(connection):
    aux = {i['to']:i for i in connection}
    return aux


def encode_conn_from_dict(dict_conn):
    return list(dict_conn.values())


def delete_connections(fluxo, to_ids):
    tmp = get_to_dict_from_conn(fluxo['connections'])
    for i in to_ids:
        try:
            del tmp[i]
        except:
            pass
    return tmp


def get_entradas(fluxo):
    entradas = [i for i in fluxo['nodes'] if i['type'] == 'entrada']
    return entradas


def get_fluxo_const(fluxo, val_dict):
    aux_fluxo = deepcopy(fluxo)
    saidas = [i for i in aux_fluxo['nodes'] if i['type'] == 'saida']
    saida_names = [i['name'] for i in saidas]
    entradas = [i for i in aux_fluxo['nodes'] if i['type'] == 'entrada']
    entrada_names = [i['name'] for i in entradas]
    ids_entradas = get_ids_entradas(entradas)
    entry_conn = get_ids_from_conn(aux_fluxo['connections'], ids_entradas)
    tmp = delete_connections(aux_fluxo, ids_entradas)
    aux_fluxo['connections'] = encode_conn_from_dict(tmp)
    encode_entradas_to_const(aux_fluxo, val_dict)
    return aux_fluxo


In [43]:
val_dict = {'entrada-1': 2, 'entrada-2': 3}
# tmp = entradas.copy()
# new_entr = encode_val_to_entradas(tmp, val_dict)

In [45]:
new = get_fluxo_const(entr, val_dict)

In [40]:
get_execution(entr, session.blocks)

nome: entrada-1 valor: 1
nome: entrada-2 valor: 2
Resultado: 3


In [46]:
get_execution(new, session.blocks)

Resultado: 5


In [50]:
file['teste']

{'nodes': [{'type': 'constante',
   'id': 'node_17205362694800',
   'name': 'constante-1',
   'options': [['valor', 4]],
   'state': {},
   'interfaces': [['saida', {'id': 'ni_17205362694801', 'value': None}]],
   'position': {'x': 23.615555172426, 'y': 173.6145673483444},
   'width': 200,
   'twoColumn': False,
   'customClasses': ''},
  {'type': 'saida',
   'id': 'node_17205362713032',
   'name': 'saida-1',
   'options': [],
   'state': {},
   'interfaces': [['saida', {'id': 'ni_17205362713043', 'value': None}]],
   'position': {'x': 311.5872730407424, 'y': 172.41468519055974},
   'width': 200,
   'twoColumn': False,
   'customClasses': ''}],
 'connections': [{'id': '17205362733416',
   'from': 'ni_17205362694801',
   'to': 'ni_17205362713043'}],
 'panning': {'x': 36.37855271680658, 'y': -60.82564451658715},
 'scaling': 0.8334151762422278}